<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=616aa86577937231fbc9b9cb4005a33d68b84ced2cb096986773855228e535de
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-30 13:44:43
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.45 C
-------------------
Today PnL: -77.23 K (-0.53%)
Current PnL: -18.73 L (-12.27%)
CY Booked + Current PnL: -6.60 L (-4.32%)
-------------------
Total profit:  2.50 L
Total loss:  -21.23 L
-------------------
Total Booked + Current PnL: 20.38 L (16.23%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.80 L (58.97%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.28,69.0,X-LC,4.55,232222.0,21856.0,6920.0,-1.04,10.39,2.98,13.68,37.0,3.16,1.61,28.63,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,107.04,59.0,M-SC,1.28,89211.0,-11566.0,11660.0,-2.23,-11.48,13.07,0.09,245.0,-0.99,0.62,17.20,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-16.77,70.0,H-MC,16.93,213551.0,33743.0,15611.0,-1.36,18.77,7.31,27.45,88.0,2.16,1.48,59.25,XR,NTT,BANKS
42,ITC,452.0,-36.66,61.0,X-LC,1.28,204598.0,4460.0,16429.0,-0.76,2.23,8.03,10.44,4.0,0.27,1.42,7.92,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-10.05,39.0,X-MC,2.50,88950.0,-1728.0,16634.0,-0.19,-1.91,18.70,16.44,101.0,-0.10,0.62,17.89,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,86.78,80.0,H-SC,2.49,162139.0,-20636.0,131706.0,11.58,-11.29,81.23,60.77,125.0,-0.16,1.13,45.22,XR,NTT,JEWELLERY
17,CAMS,5211.76,-2.73,57.0,X-SC,0.52,110634.0,8630.0,35292.0,2.48,8.46,31.90,43.06,122.0,0.24,0.77,27.02,X40N,ATH,MISC
56,RAJOOENG,143.10,-41.05,49.0,H-SC,17.11,89220.0,-13280.0,53880.0,1.48,-12.96,60.39,39.61,114.0,-0.25,0.62,8.96,AR,ATH,MISC
72,SYMPHONY,1306.00,-43.04,56.0,M-SC,17.84,121339.0,-49802.0,49749.0,1.30,-29.10,41.00,-0.03,196.0,-1.00,0.84,5.78,OX40N,NTT,DURABLES
65,SFL,1287.00,34.22,55.0,M-SC,6.00,177143.0,-85075.0,154912.0,1.29,-32.44,87.45,26.63,229.0,-0.55,1.23,10.14,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,48.36,57.0,M-MC,10.19,234106.0,9144.0,158513.0,-3.27,4.06,67.71,74.53,192.0,0.06,1.63,37.63,XY24,BTT,STEEL
84,VBL,671.64,-14.92,59.0,X-LC,8.53,306240.0,-9602.0,122251.0,-3.12,-3.04,39.92,35.67,5.0,-0.08,2.13,10.39,X40N,ATH,FMCG
36,IEX,219.00,-31.54,54.0,H-SC,16.00,195734.0,-4220.0,101449.0,-2.97,-2.11,51.83,48.63,115.0,-0.04,1.36,9.20,XR,NTT,MISC
80,UNIONBANK,163.00,-10.08,55.0,M-LC,9.57,162136.0,21296.0,23202.0,-2.41,15.12,14.31,31.59,66.0,0.92,1.13,45.61,XY24,NTT,BANKS
54,QUESS,986.00,-46.01,48.0,X-SC,37.96,53797.0,-11209.0,162133.0,-2.25,-17.24,301.38,232.18,198.0,-0.07,0.37,4.93,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.00,-15.15,44.0,M-LC,5.69,149457.0,-1488.0,110942.0,-0.50,-0.99,74.23,72.51,53.0,-0.01,1.04,4.98,XR,NTT,IT
32,HINDZINC,730.22,26.60,47.0,M-LC,11.29,206668.0,1592.0,110981.0,-1.76,0.78,53.70,54.89,52.0,0.01,1.44,25.06,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,107.04,59.0,M-SC,1.28,89211.0,-11566.0,11660.0,-2.23,-11.48,13.07,0.09,245.0,-0.99,0.62,17.20,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,52.0,H-SC,2.83,225504.0,-44163.0,80505.0,-0.84,-16.38,35.70,13.47,138.0,-0.55,1.57,14.66,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,52.0,H-SC,2.49,141197.0,-34706.0,76726.0,0.20,-19.73,54.34,23.89,149.0,-0.45,0.98,21.94,OX40N,NTT,CERAMICS
74,TATAELXSI,9161.0,-22.37,53.0,H-MC,6.44,104928.0,-26907.0,69137.0,-0.29,-20.41,65.89,32.03,98.0,-0.39,0.73,17.53,OX40N,NTT,IT
68,SIS,528.0,2033.68,45.0,H-SC,3.20,86114.0,-24918.0,48525.0,-0.69,-22.44,56.35,21.26,156.0,-0.51,0.60,16.35,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,48.36,57.0,M-MC,10.19,234106.0,9144.0,158513.0,-3.27,4.06,67.71,74.53,192.0,0.06,1.63,37.63,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,56.0,M-LC,5.18,208285.0,5115.0,37012.0,-1.78,2.52,17.77,20.74,55.0,0.14,1.45,9.55,XY25,NTT,FINANCE
32,HINDZINC,730.22,26.60,47.0,M-LC,11.29,206668.0,1592.0,110981.0,-1.76,0.78,53.70,54.89,52.0,0.01,1.44,25.06,X5K,ATH,METALS
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.46,132703.0,7406.0,51515.0,-0.96,5.91,38.82,47.03,141.0,0.14,0.92,66.16,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,55.0,M-LC,9.57,162136.0,21296.0,23202.0,-2.41,15.12,14.31,31.59,66.0,0.92,1.13,45.61,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-16.77,70.0,H-MC,16.93,213551.0,33743.0,15611.0,-1.36,18.77,7.31,27.45,88.0,2.16,1.48,59.25,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.74,129968.0,15170.0,78878.0,0.11,13.21,60.69,81.92,223.0,0.19,0.90,53.86,XR,NTT,DURABLES
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.46,132703.0,7406.0,51515.0,-0.96,5.91,38.82,47.03,141.0,0.14,0.92,66.16,MH,ATH,POWER
37,INDIAMART,4810.62,-52.96,58.0,H-SC,6.68,131180.0,7844.0,123781.0,-0.35,6.36,94.36,106.72,119.0,0.06,0.91,31.22,AR,ATH,MISC
32,HINDZINC,730.22,26.60,47.0,M-LC,11.29,206668.0,1592.0,110981.0,-1.76,0.78,53.70,54.89,52.0,0.01,1.44,25.06,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,20.40,149320.0,-127203.0,237269.0,-0.04,-46.00,158.90,39.81,54.0,-0.54,1.04,5.24,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.15,29.0,X-SC,4.92,89557.0,-39113.0,84407.0,0.19,-30.40,94.25,35.20,219.0,-0.46,0.62,2.79,X40,NTT,FOOTWEAR
24,DMART,5391.45,-18.29,32.0,X-LC,9.67,99989.0,-2597.0,29407.0,-1.49,-2.53,29.41,26.13,38.0,-0.09,0.69,22.39,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.69,37.0,X-MC,5.24,199125.0,-1222.0,31721.0,-0.41,-0.61,15.93,15.23,178.0,-0.04,1.38,31.44,X40N,NTT,REALTY
53,PGHH,17907.65,-30.81,37.0,X-MC,2.28,200415.0,-405.0,68201.0,-0.68,-0.20,34.03,33.76,80.0,-0.01,1.39,4.98,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-3.04,46.0,H-LC,1.16,155990.0,-30105.0,77589.0,-0.92,-16.18,49.74,25.51,15.0,-0.39,1.08,14.95,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,61.0,X-LC,1.28,204598.0,4460.0,16429.0,-0.76,2.23,8.03,10.44,4.0,0.27,1.42,7.92,X40,NTT,FMCG
53,PGHH,17907.65,-30.81,37.0,X-MC,2.28,200415.0,-405.0,68201.0,-0.68,-0.20,34.03,33.76,80.0,-0.01,1.39,4.98,X40,ATH,FMCG
29,HAVELLS,2069.16,-4.76,53.0,X-MC,2.97,317961.0,-10030.0,116565.0,0.31,-3.06,36.66,32.48,92.0,-0.09,2.21,8.02,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,1.02,52.0,X-MC,4.20,211830.0,20088.0,17667.0,-0.47,10.48,8.34,19.69,99.0,1.14,1.47,17.99,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.15,29.0,X-SC,4.92,89557.0,-39113.0,84407.0,0.19,-30.40,94.25,35.20,219.0,-0.46,0.62,2.79,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.78,39.0,X-MC,6.70,81530.0,-1980.0,22013.0,-1.44,-2.37,27.00,23.98,82.0,-0.09,0.57,3.31,X40,ATH,APPARELS
3,ACC,3906.00,-37.64,52.0,X-MC,2.52,186010.0,-51941.0,204592.0,-1.02,-21.83,109.99,64.15,174.0,-0.25,1.29,4.34,XY24,BTT,CEMENT
10,BAJAJHFL,181.50,-13.84,50.0,X-MC,5.89,181400.0,-19106.0,116259.0,-0.04,-9.53,64.09,48.45,90.0,-0.16,1.26,4.61,X40N,ATH,FINANCE
21,COLPAL,3767.14,-3.45,49.0,X-MC,7.66,225450.0,-37915.0,151254.0,-0.58,-14.40,67.09,43.04,84.0,-0.25,1.57,4.67,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-46.01,48.0,X-SC,37.96,53797.0,-11209.0,162133.0,-2.25,-17.24,301.38,232.18,198.0,-0.07,0.37,4.93,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.78,39.0,X-MC,6.70,81530.0,-1980.0,22013.0,-1.44,-2.37,27.00,23.98,82.0,-0.09,0.57,3.31,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.59,45.0,X-SC,4.77,82410.0,-62750.0,138680.0,-0.56,-43.23,168.28,52.31,136.0,-0.45,0.57,10.31,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,39.0,X-MC,2.50,88950.0,-1728.0,16634.0,-0.19,-1.91,18.70,16.44,101.0,-0.10,0.62,17.89,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.15,29.0,X-SC,4.92,89557.0,-39113.0,84407.0,0.19,-30.40,94.25,35.20,219.0,-0.46,0.62,2.79,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.36,52.0,X-LC,10.94,285478.0,-60478.0,127180.0,-0.67,-17.48,44.55,19.28,1.0,-0.48,1.98,5.54,X40,ATH,IT
40,INFY,2275.00,-18.52,54.0,X-LC,5.80,317604.0,4716.0,166964.0,-1.28,1.51,52.57,54.87,3.0,0.03,2.21,9.91,X40,BTT,IT
42,ITC,452.00,-36.66,61.0,X-LC,1.28,204598.0,4460.0,16429.0,-0.76,2.23,8.03,10.44,4.0,0.27,1.42,7.92,X40,NTT,FMCG
84,VBL,671.64,-14.92,59.0,X-LC,8.53,306240.0,-9602.0,122251.0,-3.12,-3.04,39.92,35.67,5.0,-0.08,2.13,10.39,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,55.0,H-LC,5.61,252888.0,-8731.0,127936.0,-0.39,-3.34,50.59,45.57,7.0,-0.07,1.76,7.61,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6966.67,56.0,L-SC,7.80,77539.0,-16271.0,92892.0,-1.19,-17.34,119.80,81.67,269.0,-0.18,0.54,50.16,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.16,80736.0,-32813.0,72864.0,-0.59,-28.90,90.25,35.27,268.0,-0.45,0.56,97.60,XR,NTT,HOTELS
50,MASFIN,398.61,-19.15,52.0,H-SC,9.16,92055.0,-5925.0,27524.0,1.14,-6.05,29.90,22.05,152.0,-0.22,0.64,33.81,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.74,129968.0,15170.0,78878.0,0.11,13.21,60.69,81.92,223.0,0.19,0.90,53.86,XR,NTT,DURABLES
37,INDIAMART,4810.62,-52.96,58.0,H-SC,6.68,131180.0,7844.0,123781.0,-0.35,6.36,94.36,106.72,119.0,0.06,0.91,31.22,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,86.78,80.0,H-SC,2.49,162139.0,-20636.0,131706.0,11.58,-11.29,81.23,60.77,125.0,-0.16,1.13,45.22,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.46,132703.0,7406.0,51515.0,-0.96,5.91,38.82,47.03,141.0,0.14,0.92,66.16,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.74,129968.0,15170.0,78878.0,0.11,13.21,60.69,81.92,223.0,0.19,0.90,53.86,XR,NTT,DURABLES
5,ANGELONE,3033.00,16.33,63.0,X-SC,5.55,205746.0,14740.0,42960.0,-0.10,7.72,20.88,30.21,157.0,0.34,1.43,30.21,X40N,NTT,FINANCE
69,SONACOMS,806.63,-32.20,70.0,M-SC,7.82,84385.0,-16774.0,56774.0,-0.20,-16.58,67.28,39.54,202.0,-0.30,0.59,18.99,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,43.97
2,50,74.94


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.18
LC    32.26
MC    24.59
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.21
XY25     12.03
XR       11.52
X40N     10.66
AR        7.84
OX40N     6.55
X200      1.77
X5K       1.44
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.54
X-LC    21.60
X-MC    18.78
M-SC    12.68
M-LC     5.06
H-LC     4.61
X-SC     4.53
H-MC     3.86
M-MC     1.63
L-SC     1.43
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.51,-3.23,36.92
IT,12.46,-17.23,80.10
FINANCE,12.13,-8.26,57.34
MISC,6.87,-16.09,80.42
ELECTRICAL,5.82,-7.85,46.09
BANKS,5.62,-5.87,60.63
PAINTS,5.51,-16.38,33.73
INSURANCE,3.77,-1.68,39.60
AUTO,2.77,-44.58,104.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3252476.0,22
XR,1228247.0,14
AR,1159925.0,9
X40,822174.0,12
X40N,624644.0,9
OX40N,541548.0,9
XY25,451280.0,8
SR,267486.0,2
X5K,110981.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3300377.0,24
M-SC,1218103.0,15
X-MC,1174813.0,14
X-LC,1028843.0,13
X-SC,527541.0,6
M-LC,282137.0,4
H-LC,274764.0,3
H-MC,261072.0,3
L-SC,255270.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1156995.0      6
           AR         831084.0      5
           XR         752664.0      7
M-SC       XY24       698116.0      6
X-MC       XY24       573360.0      4
X-LC       X40        453215.0      5
           XY24       303833.0      2
X-MC       X40        284552.0      6
X-SC       X40N       281001.0      4
H-SC       SR         267486.0      2
           OX40N      240633.0      3
H-LC       AR         205525.0      2
X-LC       X40N       195663.0      3
H-MC       XY24       176324.0      1
X-MC       XY25       168921.0      2
L-SC       XR         165756.0      2
X-SC       XY24       162133.0      1
M-MC       XY24       158513.0      1
X-MC       X40N       147980.0      2
M-SC       OX40N      142264.0      4
           XY25       128005.0      1
           XR         126402.0      2
           AR         123316.0      2
M-LC       X5K        110981.0      1
           XR         110942.0      1
L-SC       OX40N       89514.0      1
X-SC       X40         84407.0      1
X-LC       XY25        76132.0      3
H-LC       X200        69239.0      1
H-MC       OX40N       69137.0      1
L-MC       XR          56872.0      1
H-SC       MH          51515.0      1
L-LC       XY25        41210.0      1
M-LC       XY25        37012.0      1
           XY24        23202.0      1
H-MC       XR          15611.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
